In [164]:
import re
import os
import datetime
import gzip
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
import gffutils

if os.getcwd().endswith('notebook'):
    os.chdir('..')

In [3]:
sns.set(palette='colorblind', font_scale=1.3)

## Load traits

In [8]:
species_traits_path = os.path.join(os.getcwd(), 'data/condensed_traits/condensed_species_NCBI_with_ogt.csv')
species_traits_all = pd.read_csv(species_traits_path)
len(species_traits_all)

11265

In [10]:
species_traits_all['species_taxid'] = species_traits_all['species_tax_id']

In [11]:
species_traits_all.head()

,species_tax_id,species,genus,family,order,class,phylum,superkingdom,gram_stain,metabolism,...,gc_content.stdev,coding_genes.stdev,optimum_tmp.stdev,optimum_ph.stdev,growth_tmp.stdev,rRNA16S_genes.stdev,tRNA_genes.stdev,data_source,ref_id,species_taxid
0,1416943,Lactobacillus sicerae,Lactobacillus,Lactobacillaceae,Lactobacillales,Bacilli,Firmicutes,Bacteria,NaN,anaerobic,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,campedelli,152,1416943
1,1229250,Ammoniibacillus agariperforans,Ammoniibacillus,Paenibacillaceae,Bacillales,Bacilli,Firmicutes,Bacteria,NaN,aerobic,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,corkrey,181,1229250
2,1081799,Thermomarinilinea lacunifontana,Thermomarinilinea,Anaerolineaceae,Anaerolineales,Anaerolineae,Chloroflexi,Bacteria,NaN,aerobic,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,corkrey,206,1081799
3,1323370,Caloranaerobacter ferrireducens,Caloranaerobacter,Clostridiaceae,Clostridiales,Clostridia,Firmicutes,Bacteria,NaN,anaerobic,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,corkrey,380,1323370
4,192731,Halomicronema excentricum,Halomicronema,Leptolyngbyaceae,Synechococcales,NaN,Cyanobacteria,Bacteria,NaN,aerobic,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,corkrey,406,192731


## Load NCBI species short-list

In [334]:
ncbi_species_metadata = pd.read_csv(
    os.path.join(os.getcwd(), 'data/condensed_traits/ncbi_species_final.csv'),
    parse_dates=['seq_rel_date_processed'],
)
len(ncbi_species_metadata)

2359

In [335]:
ncbi_species_metadata.head()

,assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,seq_rel_date_processed,download_url_base
0,GCA_000005825.2,PRJNA28811,SAMN02603086,NaN,representative genome,398511,79885,Bacillus pseudofirmus OF4,strain=OF4,NaN,...,15/12/2010,ASM582v2,"Center for Genomic Sciences, Allegheny-Singer ...",GCF_000005825.2,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN,2010-12-15,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...
1,GCA_000005845.2,PRJNA225,SAMN02604091,NaN,reference genome,511145,562,Escherichia coli str. K-12 substr. MG1655,strain=K-12 substr. MG1655,NaN,...,26/09/2013,ASM584v2,Univ. Wisconsin,GCF_000005845.2,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN,2013-09-26,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...
2,GCA_000006175.2,PRJNA20131,SAMN00000040,NaN,representative genome,456320,2188,Methanococcus voltae A3,strain=A3,NaN,...,03/06/2010,ASM617v2,US DOE Joint Genome Institute (JGI-PGF),GCF_000006175.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN,2010-06-03,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...
3,GCA_000006605.1,PRJNA13967,SAMEA3283089,NaN,representative genome,306537,38289,Corynebacterium jeikeium K411,NaN,NaN,...,27/06/2005,ASM660v1,Bielefeld Univ,GCF_000006605.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN,2005-06-27,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...
4,GCA_000019345.1,PRJNA19087,SAMN02604063,NaN,representative genome,505682,134821,Ureaplasma parvum serovar 3 str. ATCC 27815,strain=ATCC 27815,NaN,...,26/02/2008,ASM1934v1,TIGR,GCF_000019345.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,assembly from type material,2008-02-26,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...


In [338]:
ncbi_species_metadata.columns

Index(['assembly_accession', 'bioproject', 'biosample', 'wgs_master',
       'refseq_category', 'taxid', 'species_taxid', 'organism_name',
       'infraspecific_name', 'isolate', 'version_status', 'assembly_level',
       'release_type', 'genome_rep', 'seq_rel_date', 'asm_name', 'submitter',
       'gbrs_paired_asm', 'paired_asm_comp', 'ftp_path',
       'excluded_from_refseq', 'relation_to_type_material',
       'seq_rel_date_processed', 'download_url_base'],
      dtype='object')

## Keep traits from short-listed species only

In [12]:
species_traits = species_traits_all[
    species_traits_all['species_taxid'].isin(ncbi_species_metadata['species_taxid'].values)
].reset_index(drop=True)
len(species_traits)

2359

In [15]:
species_traits.columns

Index(['species_tax_id', 'species', 'genus', 'family', 'order', 'class',
       'phylum', 'superkingdom', 'gram_stain', 'metabolism', 'pathways',
       'carbon_substrates', 'sporulation', 'motility', 'range_tmp',
       'range_salinity', 'cell_shape', 'isolation_source', 'd1_lo', 'd1_up',
       'd2_lo', 'd2_up', 'doubling_h', 'genome_size', 'gc_content',
       'coding_genes', 'optimum_tmp', 'optimum_ph', 'growth_tmp',
       'rRNA16S_genes', 'tRNA_genes', 'gram_stain.count', 'metabolism.count',
       'pathways.count', 'carbon_substrates.count', 'sporulation.count',
       'motility.count', 'range_tmp.count', 'range_salinity.count',
       'cell_shape.count', 'isolation_source.count', 'gram_stain.prop',
       'metabolism.prop', 'pathways.prop', 'carbon_substrates.prop',
       'sporulation.prop', 'motility.prop', 'range_tmp.prop',
       'range_salinity.prop', 'cell_shape.prop', 'isolation_source.prop',
       'd1_lo.count', 'd1_up.count', 'd2_lo.count', 'd2_up.count',
       'doub

In [18]:
assert np.all(species_traits['growth_tmp'].notnull())

## Load CDS

In [103]:
cds_path_fmt = os.path.join(os.getcwd(), 'data/condensed_traits/sequences/{taxid}/{taxid}_cds_from_genomic.fna.gz')

In [104]:
idx = 110
specie_taxid = species_traits.iloc[idx]['species_taxid']
species_traits.iloc[[idx]][['species_taxid', 'species', 'phylum', 'growth_tmp', 'genome_size', 'coding_genes']]

,species_taxid,species,phylum,growth_tmp,genome_size,coding_genes
110,375,Bradyrhizobium japonicum,Proteobacteria,27.0,9075243.179,8530.667


In [105]:
with gzip.open(cds_path_fmt.format(taxid=specie_taxid), mode='rt') as f:
    record_dict = SeqIO.to_dict(SeqIO.parse(f, "fasta"))

In [106]:
r = sorted(record_dict.keys())[0]

In [107]:
cds = record_dict[r]

In [108]:
cds

SeqRecord(seq=Seq('ATGTTCGCTGATCGCCTCAAAGATTACAACCTTGCCCTAGCGACCGTGCTTCAG...TGA', SingleLetterAlphabet()), id='lcl|AP012206.1_cds_BAL05302.1_1', name='lcl|AP012206.1_cds_BAL05302.1_1', description='lcl|AP012206.1_cds_BAL05302.1_1 [locus_tag=BJ6T_00010] [protein=hypothetical protein] [protein_id=BAL05302.1] [location=123..428] [gbkey=CDS]', dbxrefs=[])

In [109]:
cds.translate()

SeqRecord(seq=Seq('MFADRLKDYNLALATVLQSVNSFELVGVGLVLDVPTDIASRRIFSNRTSASWSN...CS*', HasStopCodon(ExtendedIUPACProtein(), '*')), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[])

## Load RNA

In [112]:
rna_path_fmt = os.path.join(os.getcwd(), 'data/condensed_traits/sequences/{taxid}/{taxid}_rna_from_genomic.fna.gz')

In [113]:
with gzip.open(rna_path_fmt.format(taxid=specie_taxid), mode='rt') as f:
    rna_dict = SeqIO.to_dict(SeqIO.parse(f, "fasta"))

In [118]:
rna_id = sorted(rna_dict.keys())[20]
rna = rna_dict[rna_id]

In [119]:
rna

SeqRecord(seq=Seq('GGCGGGGTAGCTCAGCTGGTTAGAGCACGGGAATCATAATCCTGGGGTCGGGGG...CCA', SingleLetterAlphabet()), id='lcl|AP012206.1_trna_23', name='lcl|AP012206.1_trna_23', description='lcl|AP012206.1_trna_23 [locus_tag=BJ6T_24130] [product=tRNA-Met] [location=2487544..2487620] [gbkey=tRNA]', dbxrefs=[])

## Putting it together

In [120]:
def load_specie_sequences(specie_taxid):
    dna_path = os.path.join(
        os.getcwd(), 
        f'data/condensed_traits/sequences/{specie_taxid}/{specie_taxid}_genomic.fna.gz',
    )
    cds_path = os.path.join(
        os.getcwd(), 
        f'data/condensed_traits/sequences/{specie_taxid}/{specie_taxid}_cds_from_genomic.fna.gz',
    )
    rna_path = os.path.join(
        os.getcwd(), 
        f'data/condensed_traits/sequences/{specie_taxid}/{specie_taxid}_rna_from_genomic.fna.gz',
    )
    
    with gzip.open(dna_path, mode='rt') as f:
        dna = SeqIO.to_dict(SeqIO.parse(f, "fasta"))
    
    with gzip.open(cds_path, mode='rt') as f:
        cds = SeqIO.to_dict(SeqIO.parse(f, "fasta"))
        
    with gzip.open(rna_path, mode='rt') as f:
        rna = SeqIO.to_dict(SeqIO.parse(f, "fasta"))
        
    return dna, cds, rna

## Parse location information

In [ ]:
dna_dict, cds_dict, rna_dict = load_specie_sequences(specie_taxid)

In [161]:
cds_dict['lcl|AP012206.1_cds_BAL05308.1_7']

SeqRecord(seq=Seq('ATGTCAGTGATCGATCTATCGCGCGCCGATCCGCGCACGTTGATCGGCAAGGGA...TAG', SingleLetterAlphabet()), id='lcl|AP012206.1_cds_BAL05308.1_7', name='lcl|AP012206.1_cds_BAL05308.1_7', description='lcl|AP012206.1_cds_BAL05308.1_7 [locus_tag=BJ6T_00070] [protein=ribonucleoside-diphosphate reductase beta subunit] [protein_id=BAL05308.1] [location=complement(4970..6043)] [gbkey=CDS]', dbxrefs=[])

In [ ]:
description = (
    "lcl|CP042806.1_cds_QEE29787.1_3650 [locus_tag=FTW19_18435] "
    "[protein=peptide chain release factor 2] [exception=ribosomal slippage] "
    "[protein_id=QEE29787.1] [location=complement(join(4615525..4616595,4616597..4616665))] [gbkey=CDS]"
)

In [165]:
gff_path_fmt = os.path.join(os.getcwd(), 'data/condensed_traits/sequences/{taxid}/{taxid}_genomic.gff.gz')

In [301]:
annotations = gffutils.create_db(
    gff_path_fmt.format(taxid=specie_taxid), 
    ':memory:', 
    merge_strategy='replace',
)

In [169]:
for f in annotations.featuretypes():
    print(f)

CDS
exon
gene
rRNA
region
tRNA
tmRNA


In [269]:
c = 0
cds_lengths = []
for i, feature in enumerate(annotations.features_of_type('CDS')):
    start, end = feature.start, feature.end
    assert start < end
    cds_lengths.append((end - start) + 1)
    c += 1
    
print(c)
np.mean(cds_lengths), np.min(cds_lengths), np.max(cds_lengths)

8829


(885.5548759768943, 120, 17028)

In [302]:
for i, feature in enumerate(annotations.features_of_type('CDS')):
    length = (feature.end - feature.start) + 1
    assert length % 3 == 0, 'nope'

In [268]:
def extract_non_coding_locations(dna_dict, annotations):
    regions = sorted(dna_dict.keys())
    full_length = sum([
        len(dna_dict[region].seq)
        for region in regions
    ])
    
    all_indices = set(range(1, full_length + 1))
    coding_indices = set()
    
    for feature_type in ['CDS', 'exon']:
        for i, feature in enumerate(annotations.features_of_type(feature_type)):
            start, end, strand = feature.start, feature.end, feature.strand
            
            if strand == '-':
                prev_start = start
                start = full_length - (end - 1)
                end = full_length - (prev_start - 1)
                assert start < end
            
            coding_indices |= set(range(start, end + 1))
            
    remaining_indices = sorted(all_indices - coding_indices)
    
    non_coding_segments = []
    start, end = None, None
    for i in remaining_indices:
        if start is None:
            start = i
            end = start
        elif end + 1 != i:
            if end != start:
                non_coding_segments.append([start, end])
            else:
                non_coding_segments.append([start])
                
            start = i
            end = start
        elif i == remaining_indices[-1]:
            if start == i:
                non_coding_segments.append([start])
            else:
                non_coding_segments.append([start, i])
        else:
            end = i
        
    return remaining_indices, full_length, non_coding_segments

In [259]:
remaining_indices, full_length, non_coding_segments = extract_non_coding_locations(dna_dict, annotations)

100 * len(remaining_indices) / full_length

33.33924163475749

In [260]:
len(non_coding_segments)

4479

In [283]:
lengths = []
for s in non_coding_segments:
    if len(s) == 1:
        lengths.append(1)
    else:
        assert len(s) == 2
        start, end = s
        assert start < end, f'{start}, {end}'
        lengths.append((end - start) + 1)

In [284]:
np.mean(lengths), np.min(lengths), np.max(lengths)

(685.3476222371065, 1, 19661)

In [265]:
non_coding_segments[0], non_coding_segments[1]

([1, 122], [429, 483])

## Location field

In [364]:
def retrieve_odd_looking_location_fields(species_taxid):
    for specie_taxid in species_taxid:
        _, cds_dict, rna_dict = load_specie_sequences(specie_taxid)
        data = {}
        data.update(cds_dict)
        data.update(rna_dict)

        keys = sorted(data.keys())

        for key in keys:
            description = data[key].description

            m1 = re.match(r'^\s?.*\[location=([0-9]+)..([0-9]+)\].*$', description)
            m2 = re.match(r'^\s?.*\[location=complement\(([0-9]+)..([0-9]+)\)\]\s?.*$', description)

            location = re.match(r'^.*\s?\[location=([^\s]+)\]\s?.*$', description)[1]
            if m1 is None and m2 is None and 'join' not in location:
                print(key, location)

In [366]:
#retrieve_odd_looking_location_fields(ncbi_species_metadata['species_taxid'].values)

In [377]:
def all_cds_have_length_modulo_3(species_taxid):
    for i, specie_taxid in enumerate(species_taxid):
        if (i+1) % 100 == 0:
            print(f'{i+1} / {len(species_taxid)}')
            
        _, cds_dict, _ = load_specie_sequences(specie_taxid)

        keys = sorted(cds_dict.keys())

        c = 0
        for key in keys:
            seq_len = len(cds_dict[key].seq)

            if seq_len % 3 != 0:
                c += 1
                
        if c > 0:
            print(specie_taxid, c, f'{100 * c / len(keys):.2f}%')

In [378]:
all_cds_have_length_modulo_3(sorted(ncbi_species_metadata['species_taxid'].values.tolist()))

24 12 0.30%
56 1 0.01%
69 301 5.91%
72 26 1.09%
122 78 1.33%
134 52 1.21%
140 12 0.96%
159 14 0.53%
162 50 1.63%
193 1 0.02%
235 99 3.00%
246 99 2.19%
254 32 0.84%
293 20 0.58%
296 120 2.58%
337 108 1.83%
346 166 3.26%
358 115 2.09%
359 71 1.37%
381 83 1.17%
382 1 0.02%
100 / 2359
426 67 1.41%
435 38 1.15%
438 34 1.21%
454 39 1.43%
476 70 2.42%
484 70 3.17%
488 98 3.69%
492 13 0.63%
519 1 0.02%
520 2 0.05%
521 1 0.03%
539 38 1.58%
545 20 0.47%
549 185 4.00%
562 30 0.69%
575 50 0.97%
615 8 0.17%
622 1 0.02%
623 141 3.00%
624 325 6.53%
630 1 0.02%
631 38 0.86%
632 62 1.47%
633 137 3.50%
636 25 0.75%
648 81 1.91%
651 60 1.33%
654 10 0.23%
668 4 0.10%
672 36 0.80%
673 12 0.34%
676 17 0.39%
687 21 0.50%
689 40 0.74%
691 7 0.16%
728 65 2.89%
729 24 1.21%
200 / 2359
735 41 2.09%
803 77 5.89%
849 15 0.94%
850 29 1.12%
859 58 2.46%
860 46 1.86%
960 2 0.05%
1017 33 1.24%
1050 33 1.24%
1075 18 0.51%
1084 1 0.03%
1112 7 0.25%
300 / 2359
1246 11 0.66%
1249 12 0.66%
1254 11 0.56%
1275 32 0.90%
1281 

290335 53 2.07%
291048 2 0.11%
292800 19 0.52%
293091 126 4.20%
293387 23 0.59%
296842 28 0.69%
299767 20 0.45%
300019 15 0.47%
303541 5 0.33%
304207 76 2.17%
311180 36 0.75%
311230 75 0.83%
312306 17 0.33%
312539 26 1.12%
314275 13 0.35%
314281 42 1.00%
317577 20 0.51%
321983 19 0.31%
321984 133 2.61%
322596 54 1.90%
323284 22 0.57%
323450 19 0.45%
1900 / 2359
327575 13 0.60%
329854 53 1.21%
330922 10 0.39%
332101 29 0.58%
333962 162 3.87%
334542 68 1.09%
336203 319 6.32%
337243 54 1.48%
338644 18 0.57%
351679 78 1.93%
353852 18 2.95%
361111 26 0.88%
371601 71 1.35%
375175 14 0.55%
376489 30 0.78%
379347 15 0.33%
386891 34 1.75%
391905 33 0.79%
399497 78 2.22%
401471 16 0.38%
402297 482 14.05%
402384 22 0.55%
412593 12 0.67%
412690 78 2.37%
2000 / 2359
413503 20 0.49%
414771 143 1.88%
414778 12 0.38%
414996 80 1.76%
416273 34 1.18%
417367 39 1.06%
417368 23 0.95%
419257 45 0.95%
419475 23 0.50%
420953 92 1.39%
421000 56 2.42%
421525 90 3.49%
421767 100 1.91%
430522 6 0.11%
436515 256 

## Percentage of non-coding DNA in species of our dataset

In [300]:
%%time
non_coding_percentages = []
for i, taxid in enumerate(species_traits['species_taxid'].values):
    if i == 0 or (i + 1) % 10 == 0:
        print(f'Specie {i+1} / {len(species_traits)}')
    
    dna_dict, _, _ = load_specie_sequences(specie_taxid)
    
    annotations = gffutils.create_db(
        gff_path_fmt.format(taxid=taxid), 
        ':memory:', 
        merge_strategy='replace',
    )
    
    remaining_indices, full_length, _ = extract_non_coding_locations(dna_dict, annotations)
    perc = 100 * len(remaining_indices) / full_length
    non_coding_percentages.append(perc)

Specie 1 / 2359


KeyboardInterrupt: 

In [381]:
seq_record = cds_dict['lcl|AP012206.1_cds_BAL05302.1_1']
seq_record

SeqRecord(seq=Seq('ATGTTCGCTGATCGCCTCAAAGATTACAACCTTGCCCTAGCGACCGTGCTTCAG...TGA', SingleLetterAlphabet()), id='lcl|AP012206.1_cds_BAL05302.1_1', name='lcl|AP012206.1_cds_BAL05302.1_1', description='lcl|AP012206.1_cds_BAL05302.1_1 [locus_tag=BJ6T_00010] [protein=hypothetical protein] [protein_id=BAL05302.1] [location=123..428] [gbkey=CDS]', dbxrefs=[])

In [382]:
seq = seq_record.seq

In [387]:
translated_seq_str = str(seq.translate())

In [388]:
translated_seq_str[-1] == '*'

True

In [389]:
'*' not in translated_seq_str[:-1]

True

In [391]:
for specie_taxid in species_traits['species_taxid'].values:
    dna_path = os.path.join(
        os.getcwd(), 
        f'data/condensed_traits/sequences/{specie_taxid}/{specie_taxid}_genomic.fna.gz',
    )
    with gzip.open(dna_path, mode='rt') as f:
        dna = SeqIO.to_dict(SeqIO.parse(f, "fasta"))
    
    if len(dna) > 1:
        print(specie_taxid)
        break

1441386


In [392]:
dna_path_1441386 = os.path.join(
    os.getcwd(), 
    f'data/condensed_traits/sequences/1441386/1441386_genomic.fna.gz',
)
with gzip.open(dna_path_1441386, mode='rt') as f:
    dna_1441386 = SeqIO.to_dict(SeqIO.parse(f, "fasta"))

In [393]:
dna_1441386.keys()

dict_keys(['AP019551.1', 'AP019552.1', 'AP019553.1'])